In [33]:
import requests
#import pandas as pd


from pymongo import MongoClient

from itertools import combinations_with_replacement

import numpy as np

import random


In [34]:
def replace_multiplier_values(value):
    ''''
    Replace pokemon multipliers in type efficiency according to the Pokemon's main series.

    Parameters:
        value: normal value from the Pokemon API data
    '''

    new_pokemon_multiplier = {2.56: 4, 
                              1.6: 2,
                              1:1,
                              0.625: 0.5,
                              0.390625: 0}
    return new_pokemon_multiplier[value]

def compute_cp(attack, defense, stamina):
    '''
    Compute the Combat Points (CP) for a single Pokémon.

    The Combat Points (CP) is a measure of the Pokémon's overall strength in Pokémon GO,
    based on the attack, defense, and stamina stats, each adjusted by an individual value (IV)
    bonus of 15.

    Parameters:
    -----------
    attack : float or int
        The base attack stat of the Pokémon.
    defense : float or int
        The base defense stat of the Pokémon.
    stamina : float or int
        The base stamina (HP) stat of the Pokémon.

    Returns:
    --------
    float
        The computed Combat Points (CP) value for the Pokémon.
    
    Notes:
    ------
    This function assumes the maximum possible IV of 15 for each stat (attack, defense, stamina).
    The multiplier 0.7903 is used as part of the game's internal formula for CP calculation.
    '''
    return np.floor(((attack + 15) * np.sqrt(defense + 15) * np.sqrt(stamina + 15) * 0.7903**2) / 10)

def transform_and_load_data(json_file: list, type_against_info: list) -> dict:
    """
    Transforms a JSON file into a new dictionary containing all relevant Pokémon data for our work.

    Args:
        json_file (list): List containing the JSON response file from the API.
        

    Returns:
        dict: Dictionary containing all relevant information from Pokémon.
    """
    # Create a MongoDB client
    client = MongoClient()

    # Access the 'genetic_algo' database
    db = client.genetic_algo

    # Access the 'pokemons' collection within the 'genetic_algo' database
    collection = db.pokemons

    # Poke ID
    poke_id = 0

    for pokemon in json_file:
        
         # Stores relevant data for each pokemon in a dict
        pokemon_data = dict()
        
        # Get name
        pokemon_data['name'] = pokemon.get('names', {}).get('English', None)
        
        # Pokemon_ID
        pokemon_data['poke_id'] = str(poke_id).zfill(4)

        # Get stats safely
        stats = pokemon.get('stats', {})
        if not stats:  # Check if stats is empty
            continue  # Skip to the next iteration if stats is empty
        
        # Incrementing poke_id's variable
        poke_id += 1 

        # Writing the stats data
        pokemon_data['stamina'] = stats.get('stamina', None)
        pokemon_data['attack'] = stats.get('attack', None)
        pokemon_data['defense'] = stats.get('defense', None)

        pokemon_type_list = ['primaryType', 'secondaryType']
        for type_list in pokemon_type_list:
            
            pokemon_type  = pokemon.get(f'{type_list}')

            if pokemon_type is not None:
                pokemon_data[type_list] = pokemon_type.get('names').get('English', {})


                # iterate over all types of pokemon info
                for type, efficiency_against in type_against_info.items():
                    
                    if type != pokemon_data[type_list]:
                        pokemon_data[f'{type_list}_against_{type}'] = replace_multiplier_values(efficiency_against[pokemon_data[type_list]])
                    else:
                        pokemon_data[f'{type_list}_against_{type}'] = 0.5


        # Calculates Pokemon's CP

        pokemon_data['CP'] = compute_cp(attack = pokemon_data['attack'], defense = pokemon_data['defense'],
                                            stamina = pokemon_data['stamina'])

        collection.insert_one(pokemon_data)

try:
    r = requests.get('https://pokemon-go-api.github.io/pokemon-go-api/api/pokedex.json')
    r.raise_for_status()
except requests.exceptions.RequestException as e:
    print('Erro:', e)  


try:
    type_against_info = requests.get('https://pogoapi.net/api/v1/type_effectiveness.json')
    type_against_info.raise_for_status()
except requests.exceptions.RequestException as e:
    print('Erro:', e) 


# Get content of the response
r_content = r.json()

# Information over effectiveness 
type_against_info = type_against_info.json()

# Transforms the json file
transform_and_load_data(r_content, type_against_info)





In [35]:
import requests
try:
    r = requests.get('https://pokemon-go-api.github.io/pokemon-go-api/api/pokedex.json')
    r.raise_for_status()
except requests.exceptions.RequestException as e:
    print('Erro:', e)  


Creating 3000 combinations of pokemon teams:

1. Accessing the pokemon data in the DB:

In [36]:
client = MongoClient()
db = client['genetic_algo'] 

collection_pokemons = db['pokemons']

pokemons = list(collection_pokemons.find())



2. Creating the teams

In [ ]:
pokemons_index = np.arange(0, len(pokemons))


# Number of pokemons that can compose a team
k = 3

# Generate all combinations
combinations_with_repo = list(combinations_with_replacement(pokemons_index, k))

# Getting random teams
pokemon_teams = random.sample(combinations_with_repo, 1000)


# Getting data from the pokemons
pokemon_teams_data = [
    {f'Team {index}': [pokemons[pokemon] for pokemon in pokemon_team]}
    for index, pokemon_team in enumerate(pokemon_teams)
]



collection_teams = db['pokemon_teams']
collection_teams.insert_many(pokemon_teams_data)

InsertManyResult([ObjectId('6724168f98a0a7e637bbed3d'), ObjectId('6724168f98a0a7e637bbed3e'), ObjectId('6724168f98a0a7e637bbed3f'), ObjectId('6724168f98a0a7e637bbed40'), ObjectId('6724168f98a0a7e637bbed41'), ObjectId('6724168f98a0a7e637bbed42'), ObjectId('6724168f98a0a7e637bbed43'), ObjectId('6724168f98a0a7e637bbed44'), ObjectId('6724168f98a0a7e637bbed45'), ObjectId('6724168f98a0a7e637bbed46'), ObjectId('6724168f98a0a7e637bbed47'), ObjectId('6724168f98a0a7e637bbed48'), ObjectId('6724168f98a0a7e637bbed49'), ObjectId('6724168f98a0a7e637bbed4a'), ObjectId('6724168f98a0a7e637bbed4b'), ObjectId('6724168f98a0a7e637bbed4c'), ObjectId('6724168f98a0a7e637bbed4d'), ObjectId('6724168f98a0a7e637bbed4e'), ObjectId('6724168f98a0a7e637bbed4f'), ObjectId('6724168f98a0a7e637bbed50'), ObjectId('6724168f98a0a7e637bbed51'), ObjectId('6724168f98a0a7e637bbed52'), ObjectId('6724168f98a0a7e637bbed53'), ObjectId('6724168f98a0a7e637bbed54'), ObjectId('6724168f98a0a7e637bbed55'), ObjectId('6724168f98a0a7e637bbed